In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np


from torch.utils.tensorboard import SummaryWriter
import os
from os.path import join
from pickle import load
from tqdm import tqdm
import random
import datetime


In [5]:

######### 설정 영역 ########
modelVersion = 'Dense_1st_torch'
nameDataset = 'IWALQQ_1st'
dataType = 'angle' # or moBWHT
# 데이터 위치
relativeDir = '../../preperation/SAVE_dataSet'
dataSetDir = join(relativeDir,nameDataset)
# 모델 저장 위치
SaveDir = '/restricted/projectnb/movelab/bcha/IMUforKnee/trainedModel/'

# tensorboard 위치
totalFold = 5

epochs = 1000

lreaningRate = 0.001
batch_size = 32

log_interval = 10
############################
# 시간 설정
time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# CPU or GPU?
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# 모델 만들기
class Mlp(nn.Module):
    def __init__(self):
        super(Mlp, self).__init__()
        self.flatten = nn.Flatten()
        self.layer1 = nn.Linear(4242,6000)
        self.layer2 = nn.Linear(6000,4000)
        self.layer3 = nn.Linear(4000,303)
        self.dropout1 = nn.Dropout(p=0.5)
        
    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.layer1(x))
        x = self.dropout1(x)
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# 개빠른가..?
class Dataset(torch.utils.data.Dataset): 
  def __init__(self, dataSetDir, dataType, sess, numFold):
      self.dataType = dataType # angle or moBWHT
      self.sess = sess # train or test
      self.load_Data_X = torch.from_numpy(np.load(join(dataSetDir,f"{str(numFold)}_fold_final_{sess}.npz"))[f'final_X_{self.sess}'])
      self.load_Data_Y = torch.from_numpy(np.load(join(dataSetDir,f"{str(numFold)}_fold_final_{sess}.npz"))[f'final_Y_{self.dataType}_{self.sess}'])
      self.load_Data_X = torch.squeeze(self.load_Data_X.type(torch.FloatTensor))
      self.load_Data_Y = torch.squeeze(self.load_Data_Y.type(torch.FloatTensor))
  def __len__(self):
      return len(self.load_Data_X)

  def __getitem__(self, idx):
    X = self.load_Data_X[idx]
    Y = self.load_Data_Y[idx]
    return X, Y

def ensure_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)


# batch당 총 error를 누적해서 줌
# TL = Total Loss per batch
def nRMSE_Axis_TLPerbatch(pred, target,axis,load_scaler4Y):
    dict_axis = {
    'x': 0,
    "y": 1,
    "z": 2,
    }
    axis = dict_axis[axis]
    nRMSE_perbatch = 0
    # 필요한 sclaer 불러오기
    
    batchNum = len(target)
    for bat in range(batchNum): # batch 내를 순회
        pred_axis   = torch.transpose(torch.reshape(torch.squeeze(pred[bat]), [3,-1]), 0, 1)[:,axis]
        target_axis = torch.transpose(torch.reshape(torch.squeeze(target[bat]), [3,-1]), 0, 1)[:,axis]
        pred_axis = (pred_axis - torch.tensor(load_scaler4Y.min_[axis])) / torch.tensor(load_scaler4Y.scale_[axis])
        target_axis = (target_axis - torch.tensor(load_scaler4Y.min_[axis])) / torch.tensor(load_scaler4Y.scale_[axis])
        nRMSE = 100 * torch.sqrt(torch.mean(torch.square(pred_axis - target_axis))) / (torch.max(target_axis) - torch.min(target_axis))
        nRMSE_perbatch += nRMSE
    return nRMSE_perbatch
    
def ensure_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)

layout = {
    "CBD": {
        "loss(MAE)": ["Multiline", ["loss(MAE)/training", "loss(MAE)/validation"]],
        "X_nRMSE": ["Multiline", [f'{dataType}_X_nRMSE/training', f"{dataType}_X_nRMSE/validation"]],
        "Y_nRMSE": ["Multiline", [f'{dataType}_Y_nRMSE/training', f"{dataType}_Y_nRMSE/validation"]],
        "Z_nRMSE": ["Multiline", [f'{dataType}_Z_nRMSE/training', f"{dataType}_Z_nRMSE/validation"]],
    },
}


In [31]:
for numFold  in range(totalFold):
    print(f'now fold: {numFold}')
    # 매 fold마다 새로운 모델
    my_model = Mlp()
    my_model.to(device)
    

    # loss function and optimizer define
    criterion = nn.L1Loss() # mean absolute error
    optimizer = torch.optim.NAdam(my_model.parameters(),lr=lreaningRate)

    angle_train = Dataset(dataSetDir, dataType, 'train',numFold)
    angle_test  = Dataset(dataSetDir, dataType, 'test', numFold)
    train_loader = DataLoader(angle_train, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(angle_test, batch_size=batch_size, shuffle=True)

    # 시각화를 위한 tensorboard 초기화
    writer = SummaryWriter(f'./logs/pytorch/{time}/{modelVersion}/{nameDataset}/{numFold}_fold')
    writer.add_custom_scalars(layout)
    x = torch.rand(1, 4242, device=device)
    writer.add_graph(my_model,x)
    # 학습시작
    load_scaler4Y = load(open(join(dataSetDir,f"{numFold}_fold_scaler4Y_{dataType}.pkl"), 'rb'))
    for epoch in range(epochs):
        my_model.train()
        
        train_loss = 0
        train_x_nRMSE = 0
        train_y_nRMSE = 0
        train_z_nRMSE = 0
        for batch_idx, (data, target) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = my_model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * data.size(0) # 이것은 모든 배치의 크기가 일정하지 않을 수 있기 때문에 이렇게 수행함! train_loss는 total loss of batch가 됨
            
            train_x_nRMSE += nRMSE_Axis_TLPerbatch(output,target, 'x', load_scaler4Y) # 해당 배치에서의 총 loss의 합
            dict_axis = {
            'x': 0,
            "y": 1,
            "z": 2,
            }
            axis = 'z'
            axis = dict_axis[axis]
            nRMSE_perbatch = 0
            # 필요한 sclaer 불러오기
            
            batchNum = len(target)
            for bat in range(batchNum): # batch 내를 순회
                pred_axis   = torch.transpose(torch.reshape(torch.squeeze(output[bat]), [3,-1]), 0, 1)[:,axis]
                check_pred_axis = pred_axis
                target_axis = torch.transpose(torch.reshape(torch.squeeze(target[bat]), [3,-1]), 0, 1)[:,axis]
                pred_axis = (pred_axis - torch.tensor(load_scaler4Y.min_[axis])) / torch.tensor(load_scaler4Y.scale_[axis])
                target_axis = (target_axis - torch.tensor(load_scaler4Y.min_[axis])) / torch.tensor(load_scaler4Y.scale_[axis])
                nRMSE = 100 * torch.sqrt(torch.mean(torch.square(pred_axis - target_axis))) / (torch.max(target_axis) - torch.min(target_axis))
                nRMSE_perbatch += nRMSE
                break
            break
        break
    break


now fold: 0


Epoch 1/1000:   0%|          | 0/23 [00:01<?, ?it/s]


In [32]:
bat

0

In [35]:
output[bat]

tensor([ 0.0592, -0.0363, -0.0942, -0.0024, -0.0205, -0.1362, -0.0237,  0.1106,
         0.0923, -0.0117, -0.0654,  0.0641,  0.0298,  0.0211, -0.1087, -0.0233,
        -0.0848,  0.0562,  0.0779,  0.0346, -0.0906, -0.0442,  0.1078,  0.0491,
         0.0055,  0.0981, -0.0577, -0.1238, -0.0424, -0.0169,  0.0256, -0.0336,
        -0.0697,  0.0233,  0.1024,  0.0869,  0.0695, -0.0568,  0.0842, -0.0024,
        -0.0100, -0.0015, -0.0037,  0.0398, -0.0024, -0.0177, -0.0262,  0.0400,
         0.0735, -0.0215,  0.1167,  0.0629, -0.0889,  0.1054, -0.0488,  0.0102,
         0.0134,  0.0640, -0.0297, -0.0367,  0.1639,  0.0809,  0.1413,  0.0039,
        -0.0779, -0.0200,  0.1038, -0.1533, -0.0730, -0.0290, -0.0225, -0.0384,
        -0.0351,  0.0494, -0.0738,  0.0561,  0.0312,  0.0980, -0.0529, -0.0670,
         0.0809, -0.0676, -0.0455,  0.0523, -0.0259, -0.1318,  0.0432,  0.1121,
        -0.1229,  0.0329,  0.0676, -0.0259,  0.0376, -0.0110, -0.0368, -0.0411,
        -0.0339,  0.0221,  0.0716,  0.00

In [34]:
check_pred_axis

tensor([-0.1093, -0.0403, -0.0168,  0.1037, -0.1326,  0.0355, -0.0253,  0.1287,
         0.0159, -0.0025,  0.1412, -0.0042,  0.1476, -0.0181,  0.1707,  0.0145,
         0.0283,  0.0569,  0.0240,  0.0639,  0.0211, -0.0272,  0.0938,  0.0456,
         0.0052,  0.1568, -0.0759,  0.0395, -0.0056, -0.1240,  0.0708, -0.1736,
         0.0805, -0.1164, -0.0665, -0.0064, -0.0715,  0.0151, -0.0010,  0.0377,
        -0.0930,  0.0296, -0.0193,  0.0102,  0.0751,  0.0021,  0.0857,  0.0393,
        -0.0151,  0.0446, -0.0877, -0.0287, -0.0412, -0.0492,  0.1107,  0.0836,
        -0.0277,  0.0219,  0.0446, -0.1755,  0.0089,  0.0932, -0.0237, -0.0030,
         0.1073, -0.0420, -0.0406,  0.0816,  0.0349, -0.0021, -0.0930, -0.0628,
        -0.0032,  0.0289, -0.0450, -0.1072, -0.0020, -0.0546, -0.0608, -0.0053,
        -0.1024, -0.0210, -0.0108, -0.0780, -0.0055,  0.0216, -0.0414, -0.0285,
         0.0702,  0.0153,  0.0353,  0.0760, -0.0552, -0.1317, -0.0077,  0.0529,
        -0.0773, -0.1022,  0.0182,  0.07

101

In [ ]:


        train_loss /= len(train_loader.sampler) 
        train_x_nRMSE /= len(train_loader.sampler) 
        train_y_nRMSE /= len(train_loader.sampler) 
        train_z_nRMSE /= len(train_loader.sampler) 
        writer.add_scalar('loss(MAE)/training', train_loss, epoch)
        writer.add_scalar(f'{dataType}_X_nRMSE/training', train_x_nRMSE, epoch)
        writer.add_scalar(f'{dataType}_Y_nRMSE/training', train_y_nRMSE, epoch)
        writer.add_scalar(f'{dataType}_Z_nRMSE/training', train_z_nRMSE, epoch)

        test_loss = 0
        test_x_nRMSE = 0
        test_y_nRMSE = 0
        test_z_nRMSE = 0
        my_model.eval()  # batch norm이나 dropout 등을 train mode 변환
        with torch.no_grad():  # autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = my_model(data)
                loss = criterion(output,target)
                test_loss += loss.item() * data.size(0)
                test_x_nRMSE += nRMSE_Axis_TLPerbatch(output,target, 'x', load_scaler4Y)# 해당 배치에서의 총 loss의 합
                test_y_nRMSE += nRMSE_Axis_TLPerbatch(output,target, 'y', load_scaler4Y) # 해당 배치에서의 총 loss의 합
                test_z_nRMSE += nRMSE_Axis_TLPerbatch(output,target, 'z', load_scaler4Y) # 해당 배치에서의 총 loss의 합             

            test_loss /= len(test_loader.sampler)
            test_x_nRMSE /= len(test_loader.sampler) 
            test_y_nRMSE /= len(test_loader.sampler) 
            test_z_nRMSE /= len(test_loader.sampler)
            writer.add_scalar('loss(MAE)/validation', test_loss, epoch)
            writer.add_scalar(f'{dataType}_X_nRMSE/validation', test_x_nRMSE, epoch)
            writer.add_scalar(f'{dataType}_Y_nRMSE/validation', test_y_nRMSE, epoch)
            writer.add_scalar(f'{dataType}_Z_nRMSE/validation', test_z_nRMSE, epoch)
        print(f'\nTrain set: Average loss: {train_loss:.4f}, X_nRMSE: {train_x_nRMSE}, Y_nRMSE: {train_y_nRMSE}, Z_nRMSE: {train_z_nRMSE}'
             +f'\nTest set: Average loss: {test_loss:.4f}, X_nRMSE: {test_x_nRMSE}, Y_nRMSE: {test_y_nRMSE}, Z_nRMSE: {test_z_nRMSE}')
    writer.close()
    dir_save_torch = join(SaveDir,modelVersion,nameDataset)
    ensure_dir(dir_save_torch)
    model_scripted = torch.jit.script(my_model) # Export to TorchScript
    model_scripted.save(join(dir_save_torch,f'{dataType}_{numFold}_fold.pt')) # Save
    # 저장된 모델 불러올 때
    # 항상 불러온 모델 뒤에 model.eval() 붙일 것!
    # https://tutorials.pytorch.kr/beginner/saving_loading_models.html#export-load-model-in-torchscript-format
    # model = torch.jit.load('model_scripted.pt')
    # model.eval()

